<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Modelado de Tópicos 

Este *cuaderno* trata sobre modelado de tópicos a partir de datos de texto. El objetivo del *cuaderno* es que usted obtenga una visión general del modelo de asignación latente de Dirichlet (LDA, por sus siglas en inglés). Busca tambien que sea capaz de crear e implementar este modelo en `Python` y que sea  capaz evaluar de interpretar los resultados e identificar el mejor modelo de tópicos para un determinado problema. 

**NO** es necesario editar el archivo o hacer una entrega. Sin embargo, los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar  libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.



## Introducción

El modelado de tópicos o temas es una faceta del procesamiento del lenguaje natural (NLP, por sus siglas en inglés). Como vimos anteriormente utilizar el lenguaje, textos, como datos puede ser extremadamente poderoso. En este *cuaderno* nos centraremos sobre el modelado de tópicos. Inmediatamente nos surge la pregunta ¿qué son los tópicos? Responderemos esa pregunta con un ejemplo. Habremos notado que en los días en que se llevan a cabo eventos importantes (como elecciones nacionales, desastres naturales o eventos deportivos), las publicaciones de las redes sociales tienden a centrarse en esos eventos. Las publicaciones de alguna manera reflejan los eventos del día, y lo hacen de diferentes maneras. Las publicaciones pueden tener, y tendrán, puntos de vista divergentes que pueden ser agrupados en clústeres de tópicos de alto nivel. Si tuviéramos tweets sobre la final del Mundial, los tópicos de esos tweets podrían cubrir puntos de vista divergentes, que van desde la calidad del arbitraje al comportamiento de los aficionados. En Estados Unidos, el presidente realiza un discurso anual entre mediados y finales de enero llamado Estado de la Unión. Con un número suficiente de publicaciones en las redes sociales, podríamos inferir o predecir las reacciones de alto nivel (tópicos) al discurso. Esto lo lograríamos agrupando las publicaciones usando las palabras claves contenidas en ellos. Por ejemplo la siguiente figura con un breve texto sobre ciencia de datos muestra como se pueden identificar palabras y asignarlas a tópicos.

<center>
<img src = "figs/topicos1.jpeg" alt = "topicos" style = "width: 500px;"/>
</center>


En  la figura entonces se muestra como palabras como información, predicció, estadística son asignadas al tópico de modelado; mientras que computacional, produccion y escala son asignadas al tópico de ingeniería. Los modelos de tópicos son importantes porque ofrecen la misma función para los datos textuales que las estadísticas clásicas para los datos numéricos. Es decir que proporcionan un resumen significativo de los datos. 

Los modelos de tópicos entran en la categoría de aprendizaje no supervisado porque, casi siempre, no se conocen de antemano los tópicos subyacentes de los documentos. Por lo tanto, no existe una variable que guie el aprendizaje. En términos de aprendizaje no supervisado, los modelos de tópicos se pueden pensar como parte del análisis de clusters, más específicamente a K-medias. Recordemos que con K-medias, primero se establece el número de clusters y luego el modelo asigna cada uno de los datos a uno de los clústers predeterminados. Lo mismo ocurre generalmente con los modelos de tópicos. Seleccionamos el número de tópicos al inicio y luego el modelo aísla las palabras que forman esa cantidad de tópicos. Este es un excelente punto de partida para una descripción general de modelado de tópicos de alto nivel.


Los modelos de tópicos buscan encontrar patrones comunes en el texto en el sentido de que los documentos descrien tópicos similares. Es decir, estos modelos identifican los tópicos abstractos en una colección de documentos (también referidos como corpus), utilizando las palabras contenidas en los documentos.  Para ello asumen que las palabras en el mismo documento están relacionadas y usan esa suposición para definir tópicos abstractos al encontrar grupos de palabras que aparecen con frecuencia una cerca de otra.  Es decir, si una oración contiene las palabras salario, empleado, y reunión, podemos asumir que esa oración trata o que su tópico es el trabajo. .  

Este tipo de algoritmos por lo general tratan de primero determinar el número de tópicos, luego identificar palabras o frases concurrentes en los documentos. A partir de esto buscar clusters de palabras que caracterizan el documentos y finalmente retornar un conjunto de tópicos abstractos que caracterizan el corpus.

Un aspecto clave de los modelos de tópicos es que no producen tópicos específicos de una palabra o una frase, sino conjunto de palabras, cada una de las cuales representa un tópico abstracto. Esto se debe a que los modelos de tópicos entienden la proximidad de las palabras, no el contexto. Por ejemplo en la figura siguiente, el modelo no tiene idea de lo que significan ala, elevar, piloto, equipaje, pasajer, o mosca; sólo sabe que estas palabras, generalmente, siempre que aparecen, aparecen muy próximas entre sí. Será nuestra tarea darle una interpretación (o no) a este tópico.

<center>
<img src = "figs/topicos3.jpeg" alt = "topicos3" style = "width: 500px;"/>
</center>

Existen varios algoritmos de modelado de tópicos, pero quizás el más conocidos es el de Asignación latente de Dirichlet, o Latent Dirichlet Allocation (LDA) en inglés. En este *cuaderno* nos centraremos en este.

## Asignación latente de Dirichlet (LDA)

En 2003, David Blei, Andrew Ng, y Michael Jordan publicaron un artículo sobre el algoritmo del modelado de tópicos conocido como Latent Dirichlet Allocation (LDA). LDA es un modelo probabilístico generativo, esto significa que el proceso de modelado comienza con el texto y funciona como ingeniería reversa a través del proceso que suponemos que lo generó, con el fin de identificar los parámetros de interés. En este caso, son los tópicos que generaron los datos que son de interés. 

Esencialmente LDA es una técnica de clustering que puede ser aplicada a colecciones de datos discretos como los son los documentos de texto. LDA es un modelo bayesiano jerárquico de tres niveles en donde cada elemento o palabra de un texto se modela como una mezcla finita de tópicos. A su vez, cada tópico se modela como una combinación infinita de palabras. 

Esta técnica de aprendizaje no supervisado se diferencia de las técnicas de clustering estudiadas anteriormente porque en este caso cada observación pertenece a más de un grupo, donde la pertenencia de un elemento a un grupo se modela como una distribución de probabilidades.

### Set-up del modelo

LDA representa los documentos como una mezcla de tópicos que generan palabras con ciertas probabilidad. Assume que los documentos se generan siguiendo un proceso definido. Al empezar a escribir un documento:

1. Decidimos el número de palabras que el documento tendrá, que surge de una distrución de Poisson.
2. Elegimos la mezcla de tópicos del documendo, esta mezcla surge de una distribución de Dirichlet sobre un conjunto fijo de K tópicos. Por ejemplo, siguiendo del ejemplo anterior, podríamos elegir que el documento consista 1/3 sobre aviones y 2/3 sobre automóviles. Intuitivamente, cuando utilizamos la distribución de Dirichlet estamos asumiento que los documentos dentro del corpus se distribuirían a lo largo del símplex en donde cada vértice se representa un tópico. Luego, cada documento se ubicaría más cercano a los vértices que representan los tópicos contenidos en él. Por ejemplo, supongamos que tenemos 7 documentos y tres tópicos posibles (aviones, automóviles y barcos), podríamos representar los documentos dentro del símplex de la siguiente manera:

<center>
<img src = "figs/Triangulo.png" alt = "LDA" style = "width: 500px;"/>
</center>

De esta manera podríamos ver que cada documento es una combinación de tópicos. El documento 1 sería 100% sobre aviones, el documento 2 sería 50% sobre aviones y 50% sobre automóviles, el documento 3 sería 100% sobre barcos, etc.


3. Generamos cada palabra en el documento siguiendo el siguiente esquema:
    
    3.1. Elegimos un tópico, de acuerdo a la distribución  multinomial que sampleamos en el paso anterior, por ejemplo, podemos elegir el tópico de avions con probabilidad 1/3 y el tópico de automóviles con 2/3.
    
    3.2 Usando el tópico generamos la palabra  (de acuerdo a la distribución multinomiál). Por ejemplo, si seleccionamos el tópico de aviones, podriamos generar la palabra "piloto" con probabilidad del 20%, y "equipaje" con probabilidad del 10%, y asi sucesivamente.


Formalmente, definimos una palabra como un item de un vocabulario indexado por $\{1, \cdots, V\}$. Las palabras se representan  mediante vectores de base uno, es decir, un vector donde sólo un elemento es 1 y el resto son 0. Así, usando superíndices para denotar componentes, la v-ésima palabra en el vocabulario se representa mediante un V-vector $w$ tal que $w^v = 1$ y $w^u = 0$ para $u\neq v$. Un *documento* es una secuencia de $N$ palabras denotadas por $\mathbf{w}=(w_1,w_2,\cdots,w_N)$, en donde $w_n$ es la n-ésima palabra de la secuencia. Un *corpus* es una colección de $M$ documentos dentodas por $\mathbf{D}=(\mathbf{w_1},\mathbf{w_2},\cdots,\mathbf{w_m})$.

Asi, para cada documento del corpus $D$, LDA supone los siguientes pasos que generan  cada documento:

1. Decidimos $N\sim Poisson(\xi)$, donde $N$ son palabras del documento que surgen de un proceso de Poisson  con parámetro $\xi$
2. Elegimos  $\theta\sim Dir(\alpha)$ donde $\theta$  es la distribución de tópicos que asumimos surgen de una distribución Dirichlet con $K$ categorias.
3. Generamos $N$ palabras, $w_n$:

    3.1. Elegimos un tópico $z_n\sim Multinomial(\theta)$, de una distribución multiomial con parámetro $\theta$
  
    3.2 Usando el tópico generamos $w_n$ de $p(w_n|z_n, \beta)$


Estos tres pasos se repiten para cada documento en el corpus.

Es importante que notemos que este modelo inicial cuenta con algunas simplificaciones:   
  - En primer lugar, la dimensionalidad $K$ de la distribución Dirichlet (y por consiguiente la dimensionalidad de la variables de los tópicos $z$) se supone fija y conocida. 
  - Segundo, las probabilidades de cada palabra son parametrizadas por una matriz $\beta$ de tamaño $k\times V$ en donde $\beta_{ij}=p(w^j = 1|z^i = 1)$, que trataremos como una cantidad fija que será estimada. 

Tomando como dados los parámetros $\alpha$ y $\beta$, la distribución de probabilidad conjunta de una mezcla de tópicos $\theta$, un conjunto de $N$ temas $\mathbf{z}$ y un conjunto de $N$ palabras $\mathbf{w}$ esta dada por:

$$p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta) = p(\theta|\alpha)\prod_{n=1}^N p(z_n|\theta)p(w_n|z_n, \beta)$$

Notemos que el lado izquierdo de esta ecuación ($p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta)$) corresponde a la probabilidad de que un documento $x$ aparezca en nuestro corpus. De este modo, nuestro objetivo será estimar $\alpha$ y $\beta$ de modo que se maximice la probabilidad de encontrar nuestra muestra de documentos.

Podemos representar la solución LDA a través de una gráfica, que nos permitirá entender un poco mejor la intuición detrás del problema: 



<center>
<img src = "figs/topicos4.jpeg" alt = "LDA1" style = "width: 500px;"/>
</center>


Las cajas son "placas" que utilizas  para representar los dos pasos iterativos del proceso. Dado que el proceso lo ejecutamos para cada documento del corpus, la placa externa (etiquetada como M) representa la iteración sobre cada documento. Mientras que , la iteración sobre palabras en el Paso 3 está representada por la placa  interna del diagrama, etiquetada como N. Los círculos representan los parámetros, las distribuciones y los resultados. El círculo etiquetado como W es la palabra seleccionada, que es el único dato conocido y, como tal, se utiliza para realizar "ingeniería inversa" en el proceso. Además de W, las otras cuatro variables en el diagrama se definen de la siguiente manera:

- $\alpha$: hiperparámetro del documento del tópico para la distribución de Dirichlet.
- $\beta$: distribución de palabras para cada tópico.
- $z$: variable latente del tópico.
- $\theta$: variable latente para la distribución de tópicos de cada documento.

$\alpha$ y $\beta$ controlan la frecuencia de tópicos de los documentos y la frecuencia de palabra en los tópicos. Si $\alpha$ aumenta, los documentos se vuelven cada vez más similares a medida que aumenta el número de tópicos en cada documento. Por otro lado, si $\alpha$ disminuye, los documentos se vuelven cada vez más disímiles a medida que disminuye el número de tópicos en cada documento. 

El parámetro $\beta$  se comporta de manera similar. Si $\beta$ aumenta, se usan más palabras del documento para modelar un tópico, mientras que un valor más bajo hace que se use una cantidad menor de palabras para un tópico. Dada la complejidad de las distribuciones en LDA, no existe una solución directa, por lo que se requiere algún tipo de algoritmo de aproximación para generar los resultados. 

### Inferencia variacional

El principal desfio de este modelo es el cálculo de la distribución posterior:

$$p(\theta, \mathbf{z}|\mathbf{w},\alpha, \beta)=\frac{p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta)}{p(\mathbf{w}|\alpha, \beta)}$$

Desafortunadamente, esta distribución no se puede calcular directamente, por ende se debe aproximar numéricamente. La inferencia variacional es uno de los algoritmos de aproximación más simples, pero tiene una derivación extensa que requiere un conocimiento significativo de la probabilidad y es material para cursos más avanzados.

Sin embargo, en este *cuaderno* describiremos la intuición detrás de un algoritmo variacional simple basado en la convexidad. 

#### Intuición


La intución detrás de la inferencia variacional es que, si la distribución real es intratable, entonces se debe encontrar una distribución más simple, llamémosla distribución variacional, muy cercana a la distribución verdadera, que es manejable, para que la inferencia sea posible. En otras palabras, dado que es imposible inferir la distribución real debido a su  complejidad, buscamos encontrar una distribución más simple que sea una buena aproximación de la distribución real.


<center>
<img src = "figs/topicos5.jpeg" alt = "LDA1" style = "width: 500px;"/>
</center>



La inferencia variacional es como tratar de ver animales en un zoológico lleno de gente. Los animales del zoológico están en un hábitat cerrado que, en este ejemplo, es la distribución posterior. Los visitantes en realidad no pueden ingresar al hábitat, por lo que los visitantes deben conformarse con ver el hábitat desde la posición más cercana posible, que es la aproximación posterior (es decir, la mejor aproximación del hábitat). Si hay mucha gente en el zoológico, puede ser difícil llegar a ese punto de vista óptimo. La gente generalmente comienza en la parte de atrás de la multitud. y avanza estratégicamente hacia ese punto de vista óptimo. El paso de los visitantes desde la parte trasera de la multitud al punto de vista óptimo, es el camino de optimización. La inferencia variacional es simplemente el proceso de acercarse lo mejor posible al punto deseado sabiendo que en realidad no se puede alcanzar el punto deseado.

Con la intuicón desarrollada veamos una aplicación en `Python`

## LDA en `Python`


Ilustremos ahora la implementación de LDA en `Phyton`. Para ello vamos a usar una muestra de comentarios sobre restaurantes en Bogotá que provienen del sitio web [tripadvisor](https://www.tripadvisor.com/). Comenzamos entonces cargando las librerias y las stopwords:

In [1]:
# Cargamos las librerías a utilizar
import pandas as pd
import numpy as np
import unidecode
import regex
import spacy
nlp = spacy.load("es_core_news_sm")
# Creamos una lista de stopwords
from nltk.corpus import stopwords
lista_stopwords = stopwords.words("spanish")
# Cargamos extra stop words
extra_stopwords = pd.read_csv('data/stopword_extend.csv', sep=',')
extra_stopwords=extra_stopwords['palabra'].to_list()
lista_stopwords=lista_stopwords+extra_stopwords
lista_stopwords=np.unique(lista_stopwords)

Luego los datos:

In [2]:
# Cargamos los datos 
ensayos= pd.read_csv('data/ensayos.csv', sep=',')
ensayos.head()

,texto,titulo,pagina
0,Introducción Noam Chomsky. Avram Noam Chomsky ...,Chomsky,1
1,toda su construcción intelectual. Si de algo n...,Chomsky,2
2,tales como la ya lejana Guerra de Vietnam (la ...,Chomsky,3
3,"un sistema de financiación sin parangón, que d...",Chomsky,4
4,de la lingüística y de las ciencias cognitivas...,Chomsky,5


En total tenemos más de 130 mil comentarios (los cuales representan cada una de las filas del dataframe) y 25 variables que describen el restaurante y el comentario.

In [3]:
ensayos.shape 

(269, 3)

Primero vamos a quedarnos solo con las columnas que nos interesan: `titulo_comentario` y `contenido_comentario`. Luego las unimos.

In [24]:
def text_cleaning(txt):

    out = unidecode.unidecode(txt)
    out = out.split(" ")
    out = [regex.sub("[^\\w\\s]|\n", "", i) for i in out]
    out = [regex.sub("^[0-9]*$", "", i) for i in out]
    out = [ i.lower() for i in out]
    out = [i for i in out if i not in lista_stopwords]
    out = ' '.join(out)
    out = nlp(out)
    out = [x.lemma_ for x in out]
    out = [regex.sub("él", "", i) for i in out]
    out = [i for i in out if len(i) >= 3]
    return out

In [25]:
clean = list(map(text_cleaning, ensayos['texto']))

In [26]:
print(clean[100])

['serio', 'candidatura', 'premio', 'nobel', 'freud', 'escribio', 'publico', 'magistral', 'libro', 'malestar', 'cultura', 'contribución', 'significativo', 'autor', 'pensamiento', 'contemporaneo', 'obstante', 'progresion', 'cancer', 'consecuent', 'dolor', 'seriar', 'unico', 'motivo', 'preocupacion', 'freud', 'ano', 'venidero', 'fruto', 'persecucion', 'nazi', 'freud', 'ver', 'obligado', 'abandonar', 'vién', 'refugiar ', 'bien', 'amado', 'londres', 'freud', 'llego', 'londres', 'recuperar ', 'largo', 'viaje', 'animos', 'reemprender', 'escrito', 'epocar', 'termino', 'ultimo', 'obra', 'moís', 'religion', 'monoteista', 'labor', 'terapeuto', 'poco', 'paciente', 'obstante', 'cancer', 'progreso', 'devino', 'inoperable', 'largo', 'agonia', 'soporto', 'estoicamente', 'murio', 'noche', 'septiembre', 'ceniza', 'reposar', 'golders', 'green', 'londres', 'freud', 'gestapo', 'estallido', 'segundo', 'guerra', 'mundial', 'adolf', 'hitler', 'promover', 'durisima', 'represalia', 'comunidad', 'judia', 'alemán

In [27]:
# Para aplicar LDA necesitamos construir un diccionario
import gensim.corpora as corpora

id2word = corpora.Dictionary(clean)

In [28]:
# Create Corpus
texts = clean

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [29]:
?LdaMulticore

In [10]:
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [30]:
# Aplicamos LDA
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint

lda_model = LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=3, 
                        random_state=123,
                        passes=80)

In [31]:
pprint(lda_model.print_topics())

[(0,
  '0.013*"lenguaje" + 0.011*"chomsky" + 0.011*"poder" + 0.009*"linguistico" + '
  '0.006*"frase" + 0.006*"generativo" + 0.006*"ser" + 0.006*"gramatico" + '
  '0.005*"estructura" + 0.005*"decir"'),
 (1,
  '0.020*"freud" + 0.006*"poder" + 0.006*"humano" + 0.005*"hombre" + '
  '0.005*"ser" + 0.004*"primero" + 0.004*"decir" + 0.004*"mismo" + '
  '0.003*"parte" + 0.003*"caso"'),
 (2,
  '0.018*"voltaire" + 0.006*"poder" + 0.005*"hacer" + 0.005*"ser" + '
  '0.005*"hombre" + 0.004*"mismo" + 0.004*"diccionario" + 0.004*"historia" + '
  '0.003*"politico" + 0.003*"ano"')]


In [32]:
# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(lda_model, corpus, id2word)

/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/o

In [33]:
LDA_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.116711  0.050337       1        1  37.007546
1     -0.009813 -0.115938       2        1  32.641662
0     -0.106897  0.065601       3        1  30.350793, topic_info=             Term        Freq       Total Category  logprob  loglift
3292        freud  304.000000  304.000000  Default  30.0000  30.0000
6473     voltaire  318.000000  318.000000  Default  29.0000  29.0000
91       lenguaje  187.000000  187.000000  Default  28.0000  28.0000
94    linguistico  136.000000  136.000000  Default  27.0000  27.0000
69     generativo   86.000000   86.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
190         deber   46.762177   99.342150   Topic3  -5.7189   0.4389
227         mismo   54.948625  186.918469   Topic3  -5.5575  -0.0319
245       posible   37.932011   71.188206   Topic3  -5.9281   0.5628
251       primero   44.067458  145.661983   Topic3  -5.7782  -0.0032
275           ver   38.816770   92.833923   Topic3  -5.9051   0.3204

[226 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1667      3  0.923543   abstracto
7861      1  0.925900     absurdo
7861      3  0.066136     absurdo
2374      3  0.957828  adecuacion
170       1  0.519367         ano
...     ...       ...         ...
1290      2  0.062984       vivir
1290      3  0.157459       vivir
3564      2  0.958764        vién
6473      1  0.980172    voltaire
6473      3  0.018849    voltaire

[350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

### Eligiendo el número de tópicos

Los modelos generalmente tienen métricas que se pueden aprovechar para evaluar su desempeño. Los modelos de tópicos no son diferentes, aunque el rendimiento, en este caso, tiene una definición ligeramente diferente. En la regresión y la clasificación, los valores pronosticados se pueden comparar con los valores reales a partir de los cuales podemos calcular medidas claras de rendimiento.

Con los modelos de tópicos, la predicción es menos confiable, porque el modelo solo conoce las
palabras con las que se entrenó y los documentos nuevos no pueden contener ninguna de esas palabras, a pesar de presentar los mismos tópicos. Debido a esa diferencia, los modelos de tópicos se evalúan usando una métrica específica para los modelos de lenguaje, llamada perplejidad.

La perplejidad, abreviado como PP, mide el número de diferentes igualmente la mayoría
palabras probables que pueden seguir a cualquier palabra dada en promedio. Consideremos dos palabras como ejemplo: el/la y anunciar. La palabra el/la puede referirse con alta probabilidad a un gran número de palabras. Por otro lado, la palabra anunciar aunque puede seguir refiriéndose a un gran número de palabras, será en menos medida que el/la. 

La idea es que palabras que, en promedio, puedan ir seguidas de un menor número de
Igualmente más probables, son más específicas y pueden estar más estrechamente vinculadas a los tópicos. Como tal, puntuaciones más bajas de perplejidad implican mejores modelos de lenguaje. La perplejidad es muy similar a la entropía, pero la perplejidad se usa típicamente porque es más fácil de interpretar. Se puede utilizar para seleccionar el número óptimo de tópicoss. Con m siendo el número de palabras en la secuencia de palabras, la perplejidad se define como:

$$
PP=\ \hat{P}\ \left(w_1.\ \ldots,\ w_m\right)-1/m
$$

En esta fórmula, w_1.\ \ldots,\ w_mson las palabras que componen algún documento en el conjunto de datos de la prueba. La probabilidad conjunta de esas palabras, P (w_1.\ \ldots,\ w_m),\ es una medida de qué tan bien el documento de prueba encaja en el modelo existente. Probabilidades más altas sugieren una modelos más fuertes.  La probabilidad se eleva a la potencia -1/m para normalizar la puntuación por el número de palabras en cada documento y para hacer que los valores más bajos sean óptimos. En ambos cosos, estos cambios aumentan la interpretabilidad del puntaje. La puntuación de perplejidad, como la raíz del el error cuadrático medio, no es muy significativo como una métrica independiente. Tiende a utilizarse como métrica de comparación. Es decir, se construyen varios modelos para los cuales la perplejidad de las puntuaciones se calculan y comparan para identificar el mejor modelo con el cual seguir.

Como mencionamos anteriormente, LDA tiene dos entradas requeridas. La primera son los documentos en sí, y la segundo es la cantidad de tópicos. Seleccionar un número apropiado de tópico puede ser muy complicado. Un enfoque para encontrar el número óptimo de tópico es buscar en varios números de tópico y seleccionar el número de tópico que corresponde a la menor puntuación de perplejidad. En el aprendizaje automático, este enfoque se conoce como búsqueda en grilla. En el siguiente ejercicio, pondremos a trabajar la búsqueda en grilla para encontrar el número óptimo de tópicos. 

EJERCICIO 7.06: SELLECIÓN DEL NÚMERO DE TÓPICOS

En este ejercicio, usamos las puntuaciones de perplejidad para modelos LDA que se ajustan a una cantidad variable de tópicos para determinar la cantidad de tópicos con los cuales avanzar. Tengamos en cuenta que el conjunto de datos original tenía los titulares clasificados en cuatro tópicos. Veamos si este enfoque devuelve cuatro tópicos:

1. Definir una función que se ajuste a un modelo LDA en varios números de tópicos y calcular la puntuación de perplejidad. Devolver dos elementos: un DataFrame que tiene el número de tópicos con su puntaje de perplejidad y el número de tópicos con el puntaje mínimo de perplejidad como un número entero:


In [34]:
perplex=lda_model.log_perplexity(corpus, total_docs=len(corpus))

In [35]:
perplex

-8.30906490929499

In [49]:
def perplejidad_ntopicos(data, dicc,ntopics):
    output_dict = {
        "Number Of Topics": [], 
        "Perplexity Score": []
    }
    
    for t in ntopics:
        lda_model = LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=t, 
                        random_state=123,
                        passes=80)
        
        output_dict["Number Of Topics"].append(t)
        output_dict["Perplexity Score"].append(lda_model.log_perplexity(corpus, total_docs=len(corpus)))
        
    output_df = pd.DataFrame(output_dict)
    
    index_min_perplexity = output_df["Perplexity Score"].idxmin()
    output_num_topics = output_df.loc[
        index_min_perplexity,  # index
        "Number Of Topics"  # column
    ]
        
    return (output_df, output_num_topics)

In [50]:
df_perplexity, optimal_num_topics = perplejidad_ntopicos(
    corpus,id2word, 
    ntopics=[i for i in range(1, 200) if i % 2 == 0]
)

In [51]:
print(df_perplexity)

    Number Of Topics  Perplexity Score
0                  2         -8.291602
1                  4         -8.305629
2                  6         -8.398410
3                  8         -8.455108
4                 10         -8.471568
..               ...               ...
94               190         -9.311135
95               192         -9.290207
96               194         -9.295004
97               196         -9.289123
98               198         -9.287738

[99 rows x 2 columns]


In [52]:
optimal_num_topics

190

In [53]:
# Modelo LDA
lda_model_opt = LdaMulticore(corpus = corpus,
    id2word = id2word,
    num_topics = optimal_num_topics,
    random_state=123)
# Mostramos las palabras dentro de los 10 tópicos
pprint(lda_model_opt.print_topics())
doc_lda = lda_model_opt[corpus]

[(19,
  '0.013*"forma" + 0.012*"generativo" + 0.009*"lenguaje" + 0.009*"frase" + '
  '0.009*"gramatico" + 0.008*"palabra" + 0.008*"cabeza" + 0.008*"analisis" + '
  '0.008*"elemento" + 0.007*"estructura"'),
 (187,
  '0.016*"freud" + 0.010*"sexualidad" + 0.010*"humano" + 0.008*"poder" + '
  '0.007*"deseo" + 0.007*"ciudad" + 0.006*"modelo" + 0.005*"nucleo" + '
  '0.005*"comportamiento" + 0.005*"persona"'),
 (4,
  '0.012*"chomsky" + 0.011*"mente" + 0.010*"humano" + 0.010*"lenguaje" + '
  '0.009*"ser" + 0.009*"poder" + 0.009*"decir" + 0.009*"linguistico" + '
  '0.007*"regla" + 0.006*"teorico"'),
 (26,
  '0.018*"chomsky" + 0.014*"mit" + 0.011*"noam" + 0.010*"historia" + '
  '0.009*"intelectual" + 0.008*"lenguaje" + 0.008*"ano" + 0.008*"halle" + '
  '0.008*"investigacion" + 0.008*"linguistico"'),
 (155,
  '0.015*"gramatico" + 0.009*"poder" + 0.009*"sonido" + 0.009*"palabra" + '
  '0.007*"regla" + 0.007*"componente" + 0.007*"fonologico" + '
  '0.007*"linguistico" + 0.007*"ser" + 0.006*"generat

#### VISUALIZACION

La salida de los modelos LDA en Python usando sklearn puede ser difícil de interpretar en forma cruda. Como es el caso en la mayoría de los ejercicios de modelado, las visualizaciones pueden ser un gran beneficio cuando se trata de interpretar y comunicar los resultados del modelo. Una librería de Python, pyLDAvis, se integra directamente con el objeto del modelo sklearn para producir gráficos sencillos. Esta herramienta de visualización devuelve un histograma que muestra las palabras más relacionadas con cada tópico y un biplot, de uso frecuente en PCA, donde cada círculo corresponde a un tópico. A partir del biplot, sabemos la prevalencia de cada tópico en todo el corpus, lo que se refleja en el área del círculo, y la similitud de los tópicos, que se refleja en la cercanía de los círculos.


El escenario ideal es que los círculos se extiendan por toda la trama y tengan un tamaño razonable y consistente. Es decir, queremos que los tópicos sean distintos y que aparezcan uniformemente en todo el corpus. Además de los gráficos de pyLDAvis, aprovecharemos el modelo t-SNE, discutido en un capítulo anterior, para producir una representación bidimensional de la matriz tópico-documento, una matriz donde cada fila representa un documento y cada columna representa la probabilidad de ese tópico que describe el documento.

Habiendo completado el ajuste del modelo LDA, creemos algunos gráficos para ayudarnos a profundizar en los resultados.



In [54]:
# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(lda_model_opt, corpus, id2word)

/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-

/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. 

In [55]:
LDA_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
70    -0.084136 -0.067119       1        1  2.501875
99     0.011429 -0.145120       2        1  1.902649
9     -0.144837 -0.010665       3        1  1.814886
33    -0.160066  0.069987       4        1  1.793124
180   -0.141616 -0.000181       5        1  1.756624
...         ...       ...     ...      ...       ...
100   -0.010932  0.083149     186        1  0.003002
108    0.139475  0.029170     187        1  0.003002
112    0.026870  0.003389     188        1  0.003002
118   -0.011739  0.027096     189        1  0.003002
189    0.111065  0.122624     190        1  0.003002

[190 rows x 5 columns], topic_info=            Term        Freq       Total  Category  logprob  loglift
6473    voltaire  310.000000  310.000000   Default  30.0000  30.0000
3292       freud  329.000000  329.000000   Default  29.0000  29.0000
128        poder  370.000000  370.000000   Default  28.0000  28.0000
91      lenguaje  209.000000  209.000000   Default  27.0000  27.0000
17       chomsky  235.000000  235.000000   Default  26.0000  26.0000
...          ...         ...         ...       ...      ...      ...
494        capaz    0.002692   35.113027  Topic190  -6.2599   0.9378
69    generativo    0.002967   92.861424  Topic190  -6.1628   0.0623
128        poder    0.003738  370.168954  Topic190  -5.9317  -1.0893
411        decir    0.003001  190.337338  Topic190  -6.1514  -0.6439
751       mental    0.002517   54.292497  Topic190  -6.3273   0.4346

[12725 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2125     36  0.293790        a yo
6765     13  0.671186        abad
7781     13  0.286326      abadia
6512      6  0.610802  abanderado
9147     10  0.401321  abandonado
...     ...       ...         ...
1708     17  0.398770        zona
8758     13  0.331055   zoroastro
8758     41  0.331055   zoroastro
3498     23  0.224903       zweig
3498     89  0.224903       zweig

[16149 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[71, 100, 10, 34, 181, 112, 117, 80, 60, 153, 83, 144, 70, 123, 67, 51, 29, 180, 55, 115, 61, 104, 103, 11, 146, 118, 165, 38, 89, 75, 156, 54, 132, 108, 3, 58, 120, 43, 98, 127, 40, 17, 56, 185, 92, 91, 102, 162, 111, 184, 163, 161, 33, 157, 72, 2, 7, 57, 25, 178, 179, 4, 62, 16, 151, 27, 69, 23, 137, 82, 143, 173, 50, 12, 124, 49, 158, 63, 128, 116, 85, 125, 6, 18, 168, 141, 122, 155, 42, 93, 136, 134, 65, 107, 154, 59, 172, 15, 36, 169, 148, 140, 167, 138, 106, 114, 160, 94, 105, 129, 97, 74, 131, 166, 130, 142, 28, 110, 186, 84, 183, 182, 64, 150, 87, 48, 13, 177, 149, 77, 164, 66, 176, 175, 44, 95, 126, 39, 96, 22, 159, 147, 53, 86, 20, 79, 133, 45, 5, 35, 26, 9, 99, 135, 8, 152, 188, 14, 171, 31, 139, 174, 187, 170, 189, 1, 145, 121, 19, 21, 24, 30, 32, 37, 41, 46, 47, 52, 68, 73, 76, 78, 81, 88, 90, 101, 109, 113, 119, 190])

Note que puede seleccionar manualmente cada tema para ver sus términos más frecuentes y/o "relevantes", utilizando diferentes valores del parámetro $\lambda$. Esto puede ayudar cuando intenta asignar un nombre interpretable por humanos o un "significado" a cada tema.

Los valores de lambda ($\lambda$) que están muy cerca de cero mostrarán términos que son más específicos para un tema elegido. Lo que significa que verá términos que son "importantes" para ese tema específico pero no necesariamente "importantes" para todo el corpus.

Los valores de lambda que están muy cerca de uno mostrarán aquellos términos que tienen la relación más alta entre la frecuencia de los términos para ese tema específico y la frecuencia general de los términos del corpus.

In [ ]:
# Guardamos la visualización como un html. 
# Es mucho más sencillo interactuar con la gráfica desde el archivo que 
# desde el notebook
#pyLDAvis.save_html(LDA_visualization, 'visualizacion_LDA.html')

## Consideraciones Finales

Cuando debemos extraer información de una gran colección de documentos aún no vista, el modelado de tópicos es un gran enfoque, ya que proporciona información sobre la estructura subyacente de los documentos. Es decir, los modelos de tópicos encuentran agrupaciones de palabras utilizando la proximidad, no el contexto.

En este cuaderno, aprendimos cómo aplicar dos de los algoritmos de modelado de tópicos más comunes y efectivos: la asignación de Dirichlet latente y la factorización de matriz no negativa. Ahora deberíamos sentirnos cómodos limpiando documentos de texto sin formato utilizando varias técnicas diferentes; técnicas que se pueden utilizar en muchos otros escenarios de modelado. Continuaremos aprendiendo cómo convertir el corpus limpio en la estructura de datos adecuada de recuentos de palabras sin procesar o pesos de palabras por documento mediante la aplicación de modelos de bolsa de palabras.

El enfoque principal del cuaderno fue ajustar los dos modelos de tópicos, incluida la optimización de la cantidad de tópicos, la conversión de la salida en tablas fáciles de interpretar y la visualización de los resultados. Con esta información, deberíamos poder aplicar modelos de tópicos completamente funcionales para obtener valor e información para cualquier negocio.



Los modelos de tópicos se pueden usar para predecir los tópicos que pertenecen a documentos no vistos, pero si vamos a hacer predicciones, es importante reconocer que los modelos de tópicos sólo conocen las palabras que se usan para entrenarlos. Es decir, si los documentos no vistos tienen palabras que no estaban en los datos de entrenamiento, el modelo no podrá procesar esas palabras incluso si se vinculan a uno de los tópico identificados en los datos de entrenamiento. Debido a este hecho, los modelos de tópicos tienden a usarse más para el análisis exploratorio y la inferencia que para la predicción.

Cada modelo de tópicos genera dos matrices. La primera matriz contiene palabras contra tópicos. Esta enumera cada palabra relacionada con cada tópico con alguna cuantificación de la relación. Dada la cantidad de palabras que considera el modelo, cada tópico sólo se describirá con una cantidad relativamente pequeña de palabras.

Las palabras se pueden asignar a un tópico o a varios tópicos con diferentes cuantificaciones. Si las palabras se asignan a uno o varios tópicos depende del algoritmo. De manera similar, la segunda matriz contiene documentos contra tópicos. Esta asigna cada documento a cada tópico mediante alguna cuantificación de la relación de cada combinación de tópico del documento.

Cuando se analiza el modelado de tópicos, es importante reforzar continuamente el hecho de que los grupos de palabras que representan los tópicos no están relacionados conceptualmente; están relacionados solo por proximidad. La proximidad frecuente de ciertas palabras en los documentos es suficiente para definir tópicos debido a una suposición establecida anteriormente: que todas las palabras en el mismo documento están relacionadas.

Sin embargo, esta suposición puede no ser cierta o las palabras pueden ser demasiado genéricas para formar tópicos coherentes. La interpretación de tópicos abstractos implica equilibrar las características innatas de los datos de texto con las agrupaciones de palabras generadas. Los datos de texto, y el lenguaje en general, son muy variables, complejos y contextuales, lo que significa que cualquier resultado generalizado debe consumirse con cautela.

Esto no es para minimizar o invalidar los resultados del modelo. Dados documentos cuidadosamente limpios y una cantidad adecuada de tópicos, las agrupaciones de palabras, como veremos, pueden ser una buena guía sobre lo que contiene un corpus y pueden incorporarse de manera efectiva en sistemas de datos más grandes.

Ya discutimos algunas de las limitaciones de los modelos de tópicos, pero hay algunos puntos adicionales que debemos considerar. La naturaleza ruidosa de los datos de texto puede hacer que los modelos de tópico asignen palabras no relacionadas con uno de los tópicos a ese tópico en particular.

Nuevamente, consideremos la oración sobre el trabajo de antes. La palabra reunión podría aparecer en la agrupación de palabras que representa el tópico de trabajo. También es posible que la palabra larga pueda estar en ese grupo, pero la palabra larga no está directamente relacionada con el trabajo. Larga puede estar en el grupo porque aparece con frecuencia muy cerca de la palabra reunión. Por lo tanto, larga probablemente se consideraría falsamente (o espuriamente) correlacionado con el trabajo y probablemente debería eliminarse de la agrupación de tópicos, si es posible. Las palabras falsamente correlacionadas en grupos de palabras pueden causar problemas significativos cuando analizando los datos.

Esto no es necesariamente una falla en el modelo. En cambio, es una característica que, dados datos ruidosos, el modelo podría extraer peculiaridades de los datos que podrían afectar negativamente los resultados. Las correlaciones espurias podrían ser el resultado de cómo, dónde o cuándo se recopilaron los datos. Si los documentos se recopilaron sólo en una región geográfica específica, las palabras asociadas con esa región podrían vincularse incorrectamente, aunque accidentalmente, a una o varias de las agrupaciones de palabras resultantes del modelo.

Tenga en cuenta que, con palabras adicionales en el grupo de palabras, podríamos adjuntar más documentos a ese tópico de los que deberían adjuntarse. Si reducimos la cantidad de palabras que pertenecen a un tópico, ese tópico se asignará a menos documentos. Tenga en cuenta que esto no es algo malo. Queremos que cada grupo de palabras contenga solo palabras que tengan sentido para que podamos asignar los tópicos apropiados a los documentos apropiados.


Este modelo tiene algunos supuestos como:
- Cada documento es solo una colección de palabras o una "bolsa de palabras". Así, el orden de las palabras y el rol gramatical de las palabras (sujeto, predicado, verbos, ...) no se consideran en el modelo.
- Palabras como a/el/la/pero/y/o/... no contienen ninguna información sobre los "temas" y, por lo tanto, pueden eliminarse de los documentos como un paso de preprocesamiento. De hecho, podemos eliminar palabras que aparecen en al menos %80 ~ %90 de los documentos, sin perder ninguna información. Por ejemplo, si nuestro corpus contiene solo documentos médicos, palabras como humano, cuerpo, salud, etc. pueden estar presentes en la mayoría de los documentos y, por lo tanto, pueden eliminarse ya que no agregan ninguna información específica que haga que el documento se diferencie del resto. 
- Sabemos de antemano cuántos temas queremos. $k$ está predeterminado.
- Los documentos son una combinación de temas.
- Los temas son una combinación de palabras.


## Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Blei, D. M., Jordan, M. I., &; Ng, A. Y. (2003). Latent Dirichlet Allocation. JMLR.org, 3, 993–1022. https://doi.org/10.5555/944919.944937 

- Fradejas Rueda, J. M. (2020). Cuentapalabras. Estilometrıa y análisis de texto con R para filólogos.

- Murphy, K. P. (2012). Machine learning: a probabilistic perspective. MIT press.

- Patel, A. A. (2019). Hands-on unsupervised learning using Python: how to build applied machine learning solutions from unlabeled data. O'Reilly Media.

